In [65]:
import json
import pandas as pd

In [66]:
dataset = None
with open("sample.json", "r") as read_content: 
    dataset = (json.load(read_content))

In [67]:
print(dataset.keys())

dict_keys(['FL', 'DV'])


In [88]:
fl_data = dataset["FL"]
processed = []

def make_entry(case_id, petitioner_rep, respondent_rep, restraining_order_terms):
    return {
        "case_id": case_id,
        "petitioner_rep": petitioner_rep, 
        "respondent_rep": respondent_rep,
        "restraining_order_terms": restraining_order_terms,
        }

restraining_order_terms = ["RESTRAINING","ORDER","ROAH","DV-130"]

def count_occurences(haystack, needle):
    try:
        return haystack.index(needle)
    except:
        return 0
    
for idx,case in enumerate(fl_data):
    case_id = f"FL_{case}"
    case_object = fl_data[case]
    petitioners = []
    respondents = []
    for party in case_object["event"]["data"]["caseParties"]:
        if party["type"] == "Petitioner":
            petitioners.append(party)
        if party["type"] == "Respondent":
            respondents.append(party)
    petitioners.sort(key=lambda x: len(x["fullName"]), reverse=True)
    respondents.sort(key=lambda x: len(x["fullName"]), reverse=True)
    if len(petitioners) == 0 or len(respondents) == 0:
        continue
    petitioner_full_name = petitioners[0]["fullName"]
    respondent_full_name = respondents[0]["fullName"]
    attornies = {}
    for attorney in case_object["event"]["data"]['caseAttornies']:
        representing = attorney["representing"]
        attornies[representing] = attorney
    occured = 0
    for event in case_object["event"]["data"]["caseEvents"]:
        comm = event["comment"]
        comm = comm if comm is not None else ""
        comm = comm.upper()
        occurences = [count_occurences(comm, term.upper()) for term in restraining_order_terms]
        occured += (sum(occurences) != 0)
    respondent_attorney = respondent_full_name in attornies
    petitioner_attorney = petitioner_full_name in attornies


    processed.append(make_entry(case_id,petitioner_attorney,respondent_attorney,occured))
dataframe = pd.DataFrame(processed)
dataframe

,case_id,petitioner_rep,respondent_rep,restraining_order_terms
0,FL_2,True,True,0
1,FL_4,False,False,0
2,FL_6,False,False,0
3,FL_8,False,False,0
4,FL_9,False,False,0
...,...,...,...,...
3706,FL_4245,False,False,0
3707,FL_4246,True,False,0
3708,FL_4250,False,False,0
3709,FL_4252,True,False,0


In [90]:
dataframe.to_csv("processed.csv")